Импорт необходимых библиотек

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, f1_score
from sklearn.neighbors import KNeighborsClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from explainerdashboard import ClassifierExplainer, ExplainerDashboard
from IPython.core.interactiveshell import InteractiveShell

In [2]:
dataset = "https://raw.githubusercontent.com/nikitaely/eda_and_dev_tools/main/online_shoppers_intention.csv"

In [4]:
df = pd.read_csv(dataset)

In [5]:
df.drop_duplicates(inplace=True)
df = df.reset_index(drop=True)
df = df.fillna(0)

In [6]:
df.head()

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,Month,OperatingSystems,Browser,Region,TrafficType,VisitorType,Weekend,Revenue
0,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,0.0,Feb,1,1,1,1,Returning_Visitor,False,False
1,0,0.0,0,0.0,2,64.000000,0.00,0.10,0.0,0.0,Feb,2,2,1,2,Returning_Visitor,False,False
2,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,0.0,Feb,4,1,9,3,Returning_Visitor,False,False
3,0,0.0,0,0.0,2,2.666667,0.05,0.14,0.0,0.0,Feb,3,2,2,4,Returning_Visitor,False,False
4,0,0.0,0,0.0,10,627.500000,0.02,0.05,0.0,0.0,Feb,3,3,1,4,Returning_Visitor,True,False


In [7]:
numeric = ['Administrative', 'Administrative_Duration', 'Informational', 'Informational_Duration', 
           'ProductRelated', 'ProductRelated_Duration', 'BounceRates', 'ExitRates', 'PageValues']
categorical = ['VisitorType', 'Weekend', 'Month', 'TrafficType', 'Browser', 'OperatingSystems', 'Region', 'SpecialDay']

In [13]:
X = df[numeric]
y = df['Revenue'].astype(int)

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.25, random_state=42)

In [14]:
gnb = GaussianNB()

gnb.fit(Xtrain, ytrain)
prediction = gnb.predict(Xtest)
f1_score(ytest, prediction)

0.5032397408207344

In [15]:
knn = KNeighborsClassifier()

knn.fit(Xtrain, ytrain)
prediction = knn.predict(Xtest)
f1_score(ytest, prediction)

0.3943661971830986

In [17]:
cat = CatBoostClassifier()

cat.fit(Xtrain, ytrain, verbose=False)
prediction = cat.predict(Xtest)
f1_score(ytest, prediction)

0.5908558030480656

Модель гардиентного бустинга показала лучший резултат, что в общем было ожидаемо. А наивный байесовкий классификатор показал себя лучше, чем модель ближайших соседей.

In [18]:
model = KNeighborsClassifier()

params = {'n_neighbors' : np.arange(2, 20, 2),
          'weights' : ['uniform', 'distance'],
          'p' : [1, 2]}

gs = GridSearchCV(model, params, scoring='f1', cv=3, n_jobs=-1, verbose=2)
gs.fit(Xtrain, ytrain)

Fitting 3 folds for each of 36 candidates, totalling 108 fits


GridSearchCV(cv=3, estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid={'n_neighbors': array([ 2,  4,  6,  8, 10, 12, 14, 16, 18]),
                         'p': [1, 2], 'weights': ['uniform', 'distance']},
             scoring='f1', verbose=2)

In [19]:
gs.best_score_, gs.best_params_

(0.4242628947404592, {'n_neighbors': 2, 'p': 2, 'weights': 'distance'})

In [20]:
nb_classifier = GaussianNB()

params_NB = {'var_smoothing': np.logspace(0,-9, num=100)}
gs_NB = GridSearchCV(estimator=nb_classifier, 
                 param_grid=params_NB, 
                 cv=3, 
                 n_jobs=-1,
                 verbose=2, 
                 scoring='f1') 
gs_NB.fit(Xtrain, ytrain)

gs_NB.best_score_, gs_NB.best_params_

Fitting 3 folds for each of 100 candidates, totalling 300 fits


(0.5104967212661224, {'var_smoothing': 1.519911082952933e-09})

In [22]:
parameters = {#'depth'         : [1,4,7,10],
                 'learning_rate' : [0.01,0.05,0.1]}
    
gs_cat = GridSearchCV(estimator=CatBoostClassifier(verbose=False),
                     param_grid=parameters,
                     cv=3,
                     n_jobs=-1,
                     verbose=2,
                     scoring='f1')
gs_cat.fit(Xtrain, ytrain)

gs_cat.best_score_, gs_cat.best_params_

Fitting 3 folds for each of 3 candidates, totalling 9 fits


(0.6403431672799266, {'learning_rate': 0.01})

In [23]:
prediction = gs_cat.best_estimator_.predict(Xtest)
f1_score(ytest, prediction)

0.617169373549884

Перебор параметров показал улучшение целевой метрики. Градиентный бустинг, так же остался лучшей моделью.

In [36]:
X = df.drop('Revenue', axis=1)
y = df['Revenue'].astype(int)

Xtrain_full, Xtest_full, ytrain_full, ytest_full = train_test_split(X, y, test_size=0.25, random_state=42)

In [26]:
ct = ColumnTransformer([
    ('ohe', OneHotEncoder(handle_unknown="ignore"), categorical),
    ('scaling', MinMaxScaler(), numeric)
])

pipe = Pipeline([
    ('transformer', ct),
    ('model', CatBoostClassifier(verbose=False))
])

pipe.fit(Xtrain_full, ytrain_full)
prediction = pipe.predict(Xtest_full)
f1_score(ytest_full, prediction)

0.6263096623981373

Результат улучшился по сравнению с моделью без категориальных признаков и масштабирования

In [27]:
parameters = {#'model__depth'         : [1,4,7,10],
                 'model__learning_rate' : [0.01,0.05,0.1]}
    
gs_pipe = GridSearchCV(estimator=pipe,
                     param_grid=parameters,
                     cv=3,
                     n_jobs=-1,
                     verbose=2,
                     scoring='f1')
gs_pipe.fit(Xtrain_full, ytrain_full)

gs_pipe.best_score_, gs_pipe.best_params_

Fitting 3 folds for each of 3 candidates, totalling 9 fits


(0.6744037882692471, {'model__learning_rate': 0.01})

In [28]:
prediction = gs_pipe.best_estimator_.predict(Xtest_full)
f1_score(ytest_full, prediction)

0.6175115207373273

А вот перебор гиперпараметров не улучшил значение целевой метрики.

## Explainer Dashboard

In [29]:
InteractiveShell.ast_node_interactivity = "all"

In [46]:
explainer = ClassifierExplainer(gs_cat.best_estimator_, Xtest, ytest)
db = ExplainerDashboard(explainer)

Note: model_output=='probability'. For CatBoostClassifier shap values normally get calculated against X_background, but paramater X_background=None, so using X instead
Generating self.shap_explainer = shap.TreeExplainer(model, X, model_output='probability', feature_perturbation='interventional')...
Note: Shap interaction values will not be available. If shap values in probability space are not necessary you can pass model_output='logodds' to get shap values in logodds without the need for a background dataset and also working shap interaction values...
Building ExplainerDashboard..
Detected notebook environment, consider setting mode='external', mode='inline' or mode='jupyterlab' to keep the notebook interactive while the dashboard is running...
For this type of model and model_output interactions don't work, so setting shap_interaction=False...
The explainer object has no decision_trees property. so setting decision_trees=False...
Generating layout...
Calculating shap values...


100%|===================| 3051/3056 [06:50<00:00]        

Calculating prediction probabilities...
Calculating metrics...
Calculating confusion matrices...
Calculating classification_dfs...
Calculating roc auc curves...
Calculating pr auc curves...
Calculating liftcurve_dfs...
Calculating dependencies...
Calculating permutation importances (if slow, try setting n_jobs parameter)...
Calculating predictions...
Calculating pred_percentiles...
Reminder: you can store the explainer (including calculated dependencies) with explainer.dump('explainer.joblib') and reload with e.g. ClassifierExplainer.from_file('explainer.joblib')
Registering callbacks...


In [47]:
db.run()

Starting ExplainerDashboard on http://192.168.64.26:8050


ConnectionError: HTTPConnectionPool(host='0.0.0.0', port=8050): Max retries exceeded with url: /_alive_0ad08605-0bcd-41e4-95b5-a2180f58f863 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001CDC4C08B10>: Failed to establish a new connection: [WinError 10049] Требуемый адрес для своего контекста неверен'))

Самый важный признак для получения прогназа это PageValues.